In [1]:
import polars as pl

data = pl.read_csv('../data.csv')
train_data = data.filter(pl.col("dataset") == 'train').drop("dataset")
test_data = data.filter(pl.col("dataset") == 'test').drop("dataset")
train_data.write_csv('train.csv')
test_data.write_csv('test.csv')

print("Done")
train_data.head()

Done


FarmID,category,Crop,State,District,Sub-District,SDate,HDate,CropCoveredArea,CHeight,CNext,CLast,CTransp,IrriType,IrriSource,IrriCount,WaterCov,ExpYield,Season,geometry,tif_path
i64,str,str,str,str,str,str,str,i64,i64,str,str,str,str,str,i64,i64,i64,str,str,str
1326576,"""Healthy""","""Paddy""","""Telangana""","""Medak""","""Kulcharam""","""2023-11-25 0:00:00""","""2024-04-14 0:00:00""",97,54,"""Pea""","""Lentil""","""Transplanting""","""Flood""","""Groundwater""",4,87,17,"""Rabi""","""POLYGON ((78.18079255482755 17…","""downloads/20240412T050649_2024…"
1326577,"""Healthy""","""Paddy""","""Telangana""","""Medak""","""Kulcharam""","""2023-11-13 0:00:00""","""2024-04-26 0:00:00""",82,58,"""Pea""","""Lentil""","""Transplanting""","""Flood""","""Canal""",5,94,15,"""Rabi""","""POLYGON ((78.17483419891283 17…","""downloads/20240425T051649_2024…"
1326578,"""Healthy""","""Paddy""","""Telangana""","""Medak""","""Kulcharam""","""2023-12-19 0:00:00""","""2024-04-28 0:00:00""",92,91,"""Pea""","""Lentil""","""Transplanting""","""Flood""","""Canal""",3,99,20,"""Rabi""","""POLYGON ((78.16888476438905 17…","""downloads/20240427T050651_2024…"
1331836,"""Diseased""","""Paddy""","""Telangana""","""Medak""","""Kulcharam""","""2023-02-11 0:00:00""","""2024-11-04 0:00:00""",91,52,"""Pea""","""Lentil""","""Transplanting""","""Flood""","""Canal""",5,92,16,"""Rabi""","""POLYGON ((78.16821795882959 17…","""downloads/20241103T050951_2024…"
1326579,"""Diseased""","""Paddy""","""Telangana""","""Medak""","""Kulcharam""","""2023-12-12 0:00:00""","""2024-05-19 0:00:00""",94,55,"""Pea""","""Lentil""","""Transplanting""","""Flood""","""Canal""",5,97,20,"""Rabi""","""POLYGON ((78.17244459940454 17…","""downloads/20240517T050651_2024…"


In [2]:
train_data = train_data.drop("State")
test_data = test_data.drop("State")
train_data = train_data.with_columns(pl.col("category").alias("Target")).drop("category")
test_data = test_data.with_columns(pl.col("category").alias("Target")).drop("category")

In [3]:
train_data = train_data.with_columns(pl.col("SDate").str.to_date("%Y-%m-%d %H:%M:%S").alias("SDate"))
train_data = train_data.with_columns(pl.col("HDate").str.to_date("%Y-%m-%d %H:%M:%S").alias("HDate"))

test_data = test_data.with_columns(pl.col("SDate").str.to_date("%Y-%m-%d %H:%M:%S").alias("SDate"))
test_data = test_data.with_columns(pl.col("HDate").str.to_date("%Y-%m-%d %H:%M:%S").alias("HDate"))
train_data.drop("FarmID", "Crop", "District", "Sub-District").head()

SDate,HDate,CropCoveredArea,CHeight,CNext,CLast,CTransp,IrriType,IrriSource,IrriCount,WaterCov,ExpYield,Season,geometry,tif_path,Target
date,date,i64,i64,str,str,str,str,str,i64,i64,i64,str,str,str,str
2023-11-25,2024-04-14,97,54,"""Pea""","""Lentil""","""Transplanting""","""Flood""","""Groundwater""",4,87,17,"""Rabi""","""POLYGON ((78.18079255482755 17…","""downloads/20240412T050649_2024…","""Healthy"""
2023-11-13,2024-04-26,82,58,"""Pea""","""Lentil""","""Transplanting""","""Flood""","""Canal""",5,94,15,"""Rabi""","""POLYGON ((78.17483419891283 17…","""downloads/20240425T051649_2024…","""Healthy"""
2023-12-19,2024-04-28,92,91,"""Pea""","""Lentil""","""Transplanting""","""Flood""","""Canal""",3,99,20,"""Rabi""","""POLYGON ((78.16888476438905 17…","""downloads/20240427T050651_2024…","""Healthy"""
2023-02-11,2024-11-04,91,52,"""Pea""","""Lentil""","""Transplanting""","""Flood""","""Canal""",5,92,16,"""Rabi""","""POLYGON ((78.16821795882959 17…","""downloads/20241103T050951_2024…","""Diseased"""
2023-12-12,2024-05-19,94,55,"""Pea""","""Lentil""","""Transplanting""","""Flood""","""Canal""",5,97,20,"""Rabi""","""POLYGON ((78.17244459940454 17…","""downloads/20240517T050651_2024…","""Diseased"""


In [4]:
train_data = train_data.with_columns((pl.col("HDate") - pl.col("SDate")).alias("Cage").cast(pl.Int64)).drop("SDate", "HDate")
test_data = test_data.with_columns((pl.col("HDate") - pl.col("SDate")).alias("Cage").cast(pl.Int64)).drop("SDate", "HDate")

train_data.head()

FarmID,Crop,District,Sub-District,CropCoveredArea,CHeight,CNext,CLast,CTransp,IrriType,IrriSource,IrriCount,WaterCov,ExpYield,Season,geometry,tif_path,Target,Cage
i64,str,str,str,i64,i64,str,str,str,str,str,i64,i64,i64,str,str,str,str,i64
1326576,"""Paddy""","""Medak""","""Kulcharam""",97,54,"""Pea""","""Lentil""","""Transplanting""","""Flood""","""Groundwater""",4,87,17,"""Rabi""","""POLYGON ((78.18079255482755 17…","""downloads/20240412T050649_2024…","""Healthy""",12182400000
1326577,"""Paddy""","""Medak""","""Kulcharam""",82,58,"""Pea""","""Lentil""","""Transplanting""","""Flood""","""Canal""",5,94,15,"""Rabi""","""POLYGON ((78.17483419891283 17…","""downloads/20240425T051649_2024…","""Healthy""",14256000000
1326578,"""Paddy""","""Medak""","""Kulcharam""",92,91,"""Pea""","""Lentil""","""Transplanting""","""Flood""","""Canal""",3,99,20,"""Rabi""","""POLYGON ((78.16888476438905 17…","""downloads/20240427T050651_2024…","""Healthy""",11318400000
1331836,"""Paddy""","""Medak""","""Kulcharam""",91,52,"""Pea""","""Lentil""","""Transplanting""","""Flood""","""Canal""",5,92,16,"""Rabi""","""POLYGON ((78.16821795882959 17…","""downloads/20241103T050951_2024…","""Diseased""",54604800000
1326579,"""Paddy""","""Medak""","""Kulcharam""",94,55,"""Pea""","""Lentil""","""Transplanting""","""Flood""","""Canal""",5,97,20,"""Rabi""","""POLYGON ((78.17244459940454 17…","""downloads/20240517T050651_2024…","""Diseased""",13737600000


In [24]:
X = train_data.drop("FarmID", "geometry", 'tif_path', 'Target').to_pandas()
y = train_data.select("Target").to_pandas()

In [ ]:
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
import numpy as np

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

[4.16033755 0.30310483 4.06597938 4.66193853]


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['CropCoveredArea', 'CHeight', 'IrriCount', 'WaterCov', 'ExpYield',
       'Cage'],
      dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['Crop', 'District', 'Sub-District', 'CNext', 'CLast', 'CTransp',
       'IrriType', 'IrriSource', 'Season'],
      dtype='object'))])),
                ('classifier',
                 <catboost.core.CatBoostClassifier object at 0x000001D224A1F910>)])

In [51]:
from imblearn.over_sampling import SMOTENC
from imblearn.pipeline import Pipeline

# Convert categorical_features to a list of indices
categorical_features_indices = [X.columns.get_loc(col) for col in categorical_features]

smote = SMOTENC(sampling_strategy='minority', categorical_features=categorical_features_indices)
clf = Pipeline(steps=[
                      ('smote', smote),
                      ('classifier', CatBoostClassifier(iterations=1000, verbose=200, cat_features=categorical_features_indices))])
clf.fit(X_train, y_train, classifier__eval_set=(X_valid, y_valid))

c:\Users\chavv\anaconda\envs\telangana\lib\site-packages\sklearn\base.py:484: FutureWarning: `BaseEstimator._check_n_features` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_n_features` instead.
  warnings.warn(
c:\Users\chavv\anaconda\envs\telangana\lib\site-packages\sklearn\base.py:493: FutureWarning: `BaseEstimator._check_feature_names` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_feature_names` instead.
  warnings.warn(


Learning rate set to 0.113318
0:	learn: 1.2610320	test: 1.2826111	best: 1.2826111 (0)	total: 62.2ms	remaining: 1m 2s
200:	learn: 0.3921076	test: 0.7287004	best: 0.7253952 (164)	total: 13.9s	remaining: 55.4s
400:	learn: 0.3404728	test: 0.7330214	best: 0.7253952 (164)	total: 27.9s	remaining: 41.8s
600:	learn: 0.2948608	test: 0.7407453	best: 0.7253952 (164)	total: 42.5s	remaining: 28.2s
800:	learn: 0.2544913	test: 0.7496291	best: 0.7253952 (164)	total: 57.1s	remaining: 14.2s
999:	learn: 0.2208794	test: 0.7601634	best: 0.7253952 (164)	total: 1m 12s	remaining: 0us

bestTest = 0.7253952396
bestIteration = 164

Shrink model to first 165 iterations.


c:\Users\chavv\anaconda\envs\telangana\lib\site-packages\sklearn\utils\_tags.py:354: FutureWarning: The SMOTENC or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(


Pipeline(steps=[('smote',
                 SMOTENC(categorical_features=[0, 1, 2, 5, 6, 7, 8, 9, 13],
                         sampling_strategy='minority')),
                ('classifier',
                 <catboost.core.CatBoostClassifier object at 0x000001D2246CC6D0>)])

In [52]:
y_pred = clf.predict(X_valid)   
from sklearn.metrics import classification_report
print(classification_report(y_valid, y_pred))

              precision    recall  f1-score   support

    Diseased       0.00      0.00      0.00        85
     Healthy       0.82      0.98      0.89      1293
       Pests       0.00      0.00      0.00       115
    Stressed       0.08      0.04      0.05        85

    accuracy                           0.80      1578
   macro avg       0.23      0.25      0.23      1578
weighted avg       0.68      0.80      0.73      1578



c:\Users\chavv\anaconda\envs\telangana\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\chavv\anaconda\envs\telangana\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\chavv\anaconda\envs\telangana\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result)

In [58]:
# Load submission file
import polars as pl
sample_submission = pl.read_csv( '../telangana_data/data/SampleSubmission.csv')

# Predict on the test set
# test_encoded['target'] = model.predict(test_encoded.drop(columns=['FarmID', 'dataset', 'target']))  # Drop 'FarmID' if it exists in test_encoded
# test_encoded['target'] = test_encoded['target'].astype(int).map(idx_to_category_mapper)
# test_encoded_pl = pl.from_pandas(test_encoded)

# sample_submission = sample_submission.join(test_encoded_pl, left_on='ID', right_on='FarmID').select(['ID', 'target'])
X_sub = test_data.drop("FarmID", "geometry", 'tif_path', 'Target').to_pandas()
y_sub = clf.predict(X_sub)

In [67]:
y_sub.reshape(-1)

array(['Healthy', 'Healthy', 'Healthy', ..., 'Stressed', 'Stressed',
       'Healthy'], dtype=object)

In [69]:
submission = test_data.with_columns(pl.Series(name='Target',values=y_sub.reshape(-1))).select("FarmID", "Target")
submission.write_csv('submission.csv')
submission

FarmID,Target
i64,str
85197,"""Healthy"""
779677,"""Healthy"""
1331840,"""Healthy"""
483053,"""Healthy"""
542945,"""Healthy"""
…,…
551119,"""Stressed"""
916476,"""Stressed"""
121860,"""Stressed"""
